In [ ]:
from google import genai

client = genai.Client(api_key="sua api key")

In [ ]:
import spacy
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
!python -m spacy download pt_core_news_lg
nlp = spacy.load("pt_core_news_lg")
import gradio as gr
import re


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 2.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

faq = {

    "oi" : "Oi! Sou seu assistente virtual. Em que posso ajudar?",


    "ola" : "Olá! Sou seu assistente virtual. Em que posso ajudar?",



    "preciso de ajuda" : "Olá! Sou seu assistente virtual. Em que posso ajudar?",



    "Quero falar com o suporte humano": "O numero para o nosso suporte é: 19 xxxx-xxx",



    "Preciso falar com o seu suporte" : "O numero para o nosso suporte é: 19 xxxx-xxx",



    "minha máquina de lavar não está ligando": "Verifique se o cabo de energia está bem conectado e se há energia na tomada. Se o problema persistir, entre em contato com a assistência técnica.",



    "a máquina de lavar está fazendo barulho": "Certifique-se de que não há moedas, botões ou objetos soltos no tambor. Barulhos fortes também podem indicar desequilíbrio na carga.",



    "como limpar o filtro da máquina de lavar": "Desligue a máquina da tomada, abra a tampa inferior e retire o filtro girando-o no sentido anti-horário. Lave com água corrente e recoloque.",



    "a roupa sai muito molhada da máquina": "Verifique se o ciclo de centrifugação está selecionado corretamente. Cargas muito grandes ou mal distribuídas podem afetar o desempenho.",



    "como faço para usar o modo econômico da máquina": "Selecione o ciclo ‘Econômico’ ou ‘Rápido’ no painel e use a quantidade de sabão indicada no manual. Isso economiza água e energia.",



    "a máquina está com mau cheiro": "Recomenda-se fazer uma lavagem com o tambor vazio, utilizando vinagre branco ou produtos específicos para limpeza de máquinas.",



    "meu fogão não está acendendo": "Verifique se o gás está aberto e se o acendimento automático está funcionando. Caso use fósforo, veja se o queimador não está entupido.",


    "como limpar o forno do fogão": "Use um pano úmido com detergente neutro. Evite produtos abrasivos. Para sujeiras pesadas, use vinagre morno e bicarbonato.",



    "meu forno não aquece direito": "Verifique se o seletor de temperatura está funcionando e se o acendimento está completo. Caso o problema continue, pode ser falha no termostato.",



    "como saber o tamanho do botijão ideal": "Verifique o manual do seu modelo. A maioria dos fogões domésticos utiliza botijões de 13 kg com válvula padrão.",



    "quanto tempo dura o acendimento automático": "Depende do modelo, mas em média o acendimento deve ocorrer em até 3 segundos após pressionar o botão."



}


def limpar_texto(texto):
    texto = texto.lower()

    texto_sem_pontuacao = re.sub(r'[^\w\s]', '', texto)

    texto_final = ' '.join(texto_sem_pontuacao.split())

    return texto_final

def preprocessar(texto):

    doc = nlp(texto.lower())

    tokens_validos = []



    for token in doc:

        if not token.is_stop and token.is_alpha:

            tokens_validos.append(token.lemma_)



    return " ".join(tokens_validos)


faq_list = list(faq.keys())






/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:

faq_processadas = []

for pergunta in faq_list:
    texto_processado = preprocessar(pergunta)
    pergunta_sem_pontuacao = limpar_texto(texto_processado)

    faq_processadas.append(pergunta_sem_pontuacao)

faq_embeddings = model.encode(faq_processadas, convert_to_tensor=True)


def responder_mensagem(mensagem, historico=None):








    sem_pontuacao = limpar_texto(mensagem)
    usuario_limpo = preprocessar(sem_pontuacao).lower()
    usuario_embedding = model.encode(usuario_limpo, convert_to_tensor=True)

    similaridades = util.cos_sim(usuario_embedding, faq_embeddings)[0]
    indice = similaridades.argmax().item()
    melhor_pergunta = faq_list[indice]
    melhor_resposta = faq[melhor_pergunta]
    pontuacao = similaridades[indice].item()

    # Escolher resposta
    if pontuacao <= 0.64:
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=f"Você é um chatbot para a whirlpool de Rio Claro e a pergunta do cliente não se encontra dentro da FAQ, porém auxilie ele a fazer perguntas sobre a FAQ no caso: {faq_list}. empresa que no caso é a whirlpool de rio claro que produz maquina de lavar e fogao e também não precisa dar todas opções de faq ao usuario, caso a mensagem do usuario seja saudações, agradeciemntos ou despedidas apenas responda de forma breve e cordialmente a pergunta dele foi: {mensagem}"
        )
        resposta = f"{response.text}"
    else:
        resposta = melhor_resposta

    return resposta
import gradio as gr

gr.ChatInterface(
    fn=responder_mensagem,
    title="ChatBot Whirlpool 🧠",
    description="Chat de exemplo com Gradio atualizado."
).launch(share=True)




/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://830a222865336595aa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
